# Lesson 25

* last lesson of Part 2

* CLiP embeddings will not be covered, but will be covered in the next Part (Which doesn't exist and probably never will).





## Diffusion of pixels for sound. 

[Jono's Audio with Simple Diffusion](https://colab.research.google.com/drive/1b3CeZB2FfRGr5NPYDVvk34hyZFBtgub5?usp=sharing)

* Jono said much of the code was copied from course notebooks, except he is applying it to sound. 

* Data is coming from a Hugging face dataset of bird calls.

* Spectragram is a 2D representation of sound as images! 

* THe diffusers has Mel-spectragram function.  (This is a spectrogram using log scale for frequency i think)

* Note the import has changed to `form diffusers import Mel`.   

* The image conversion from the spectragram to audio is an approximation because the image has no phase information.

* *Everything Else* is basically the same process of 'simple diffusion'([this paper maybe, Dec 2023](https://arxiv.org/abs/2301.11093)), using the model in [notebook 30](https://github.com/fastai/course22p2/blob/master/nbs/30_lsun_diffusion-simple.ipynb). This will be discussed i think later in this lesson??

* Seems to work well.

* There are people doing this with music as well: [Riffusion](https://www.riffusion.com/)  and also [SUno](https://suno.com/home)

## VAE 
At 17:50
[Notebook 29](https://github.com/fastai/course22p2/blob/master/nbs/29_vae.ipynb)

* The notebook starts with a simple auto encoder similar to what we did in the beginning of this course. 

* The issue is that altough the model can learn to encode and decode the data, it doesn't learn a good representation of the data. This is because it is only making a point estimate of the latent space.
* The VAE solution is to learn an encoding with uncertainty,  into noise space, and then decode from noise space to the data space.
* This is done by learning the mean and (log) variance in latent space, and then sampling from a normal distribution with that mean and variance for decoding. 
* To construct the loss function, we need to calculate the KL divergence between the learned distribution and a normal distribution wiht mean 0 and standard deviation 1. (The prior).  This is added to the standard cross entropy loss. 
* NB: This is the way it was presented in the video, but a better way to understand it is the entire loss is the KL divergence between the true latent posterior and this gaussian approximation to the posterior of the latents.  See for example [Derivaiton of VAE loss](https://arxiv.org/abs/1907.08956) 

Very roughly, if $z$ is latents adn $x$ is data, then the (backward) KL divergence is:

$$
D_{KL}(p_{\theta}(z)||p(z|x)) = \int p_{\theta}(z) \log \frac{p_{\theta}(z)}{p(z|x)} dz
$$

Apply Bayes rule, with prior $p(z)$ and likelihood $p(x|z)$, and evidence $p(x)$:

$$
p(z \mid x) \;=\; \frac{p(x \mid z)\,p(z)}{p(x)},
$$

We have: 

 

$$
\begin{align*}
D_{KL}(p_{\theta}(z)\,\|\,p(z \mid x)) 
&= E_{z \sim p_{\theta}}\!\Biggl[\log \frac{p(x)\, p_{\theta}(z)}{p(z)\,p(x\mid z)}\Biggr] \\
&= E_{z \sim p_{\theta}}\!\Biggl[\log \frac{p_{\theta}(z)}{p(z)} \;-\; \log p(x\mid z)\Biggr] \;+\;\log p(x) \\
&= \underbrace{E_{z \sim p_{\theta}}\!\Bigl[\log \frac{p_{\theta}(z)}{p(z)}\Bigr]}_{\displaystyle D_{KL}\bigl(p_{\theta}(z)\,\|\,p(z)\bigr)} 
   \;\;-\;\;\underbrace{E_{z \sim p_{\theta}}\!\bigl[\log p(x\mid z)\bigr]}_{\displaystyle \text{NLL}} 
   \;+\;\log p(x).
\end{align*}
$$

The evididence is not effeced by the model, so if we minimize the part that doesn't include $\log p(x)$ we are minimizing the KL divergence.  This 'part' (or rather it's negative) is called the ELBO (evidence lower bound).

This shows how the VAE loss is a combination of the KL divergence (between the latent distribution and the 'prior') and the NLL.

## LSUN 

[Notebook 30](https://github.com/fastai/course22p2/blob/master/nbs/30_lsun_diffusion-latents.ipynb)
42:40 

* This introduces a bigger dataset (LSUN), in particular 256x256 color images of 'bedroom scenes'.

* Jeremy uses a pretrained VAE from hugging face (https://huggingface.co/docs/diffusers/en/api/models/autoencoderkl) to encode the images into latents. 

* He shows how you can save these latents to numpy memory mapped files, so you can load them quickly later.

* He then uses the diffusion model  we have developed to noisify and train a noise predictor on the latents themselves.  Now to generate images we just have to sample and decode.  The images look mostly pretty good. 

* I think at *this* point we have most of stable diffusion except of course the NLP part.

## Full ImageNet 
[Notebook 31](https://github.com/fastai/course22p2/blob/master/nbs/31_imgnet_latents-widish.ipynb)

This section uses the Kaggle ImageNet dataset, and trains a classifier using the latents. 
He does do some more pre-processing of the latents, including data augmentation.
Idea is to use this model as a pretrained model for the noise predictor. 
He gets 66% accuracy (and there are 1000 categories!)

This is a mission he gives the audience, to use this model as pretrained model (for the downsampling backbone) for the noise predictor.